[View in Colaboratory](https://colab.research.google.com/github/kushsharma1001/Deep-Learning/blob/master/CIFAR.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 24
compression = 0.5
dropout_rate = 0.2

In [82]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

#print Input Size
print(img_height)
print(img_width)
print(channel)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

32
32
3


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
input_new = Conv2D(num_filter, (1,1), use_bias=False, padding='same') (input)
input_new2 = Conv2D(num_filter, (3,3), use_bias=False, padding='same') (input_new)

First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input_new2)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [87]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_553 (Conv2D)             (None, 32, 32, 24)   72          input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_554 (Conv2D)             (None, 32, 32, 24)   5184        conv2d_553[0][0]                 
__________________________________________________________________________________________________
conv2d_555 (Conv2D)             (None, 32, 32, 24)   5184        conv2d_554[0][0]                 
__________________________________________________________________________________________________
batch_norm

__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 10)           6250        flatten_9[0][0]                  
Total params: 460,450
Trainable params: 451,402
Non-trainable params: 9,048
__________________________________________________________________________________________________


In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [89]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 303s 6ms/step - loss: 1.5727 - acc: 0.4183 - val_loss: 1.5405 - val_acc: 0.4711
Epoch 2/50
19200/50000 [==========>...................] - ETA: 2:27 - loss: 1.2481 - acc: 0.5463

50000/50000 [==============================] - 255s 5ms/step - loss: 1.1726 - acc: 0.5767 - val_loss: 1.1966 - val_acc: 0.6073
Epoch 3/50
45568/50000 [==========================>...] - ETA: 21s - loss: 0.9893 - acc: 0.6440

50000/50000 [==============================] - 255s 5ms/step - loss: 0.9843 - acc: 0.6465 - val_loss: 2.6375 - val_acc: 0.4407
Epoch 4/50
50000/50000 [==============================] - 255s 5ms/step - loss: 0.8766 - acc: 0.6869 - val_loss: 1.2908 - val_acc: 0.6041
Epoch 5/50
 3328/50000 [>.............................] - ETA: 3:43 - loss: 0.8320 - acc: 0.7079

50000/50000 [==============================] - 255s 5ms/step - loss: 0.7984 - acc: 0.7182 - val_loss: 1.2166 - val_acc: 0.6266
Epoch 6/50
39424/50000 [======================>.......] - ETA: 50s - loss: 0.7401 - acc: 0.7375

50000/50000 [==============================] - 255s 5ms/step - loss: 0.7338 - acc: 0.7394 - val_loss: 1.0311 - val_acc: 0.6875
Epoch 7/50
50000/50000 [==============================] - 255s 5ms/step - loss: 0.6836 - acc: 0.7588 - val_loss: 1.3414 - val_acc: 0.6268
Epoch 8/50
 1792/50000 [>.............................] - ETA: 3:49 - loss: 0.6488 - acc: 0.7701

...

50000/50000 [==============================] - 255s 5ms/step - loss: 0.2189 - acc: 0.9209 - val_loss: 0.7253 - val_acc: 0.8137
Epoch 45/50
38528/50000 [======================>.......] - ETA: 54s - loss: 0.2167 - acc: 0.9225

50000/50000 [==============================] - 255s 5ms/step - loss: 0.2204 - acc: 0.9211 - val_loss: 0.6608 - val_acc: 0.8378
Epoch 46/50
50000/50000 [==============================] - 254s 5ms/step - loss: 0.2071 - acc: 0.9268 - val_loss: 0.7902 - val_acc: 0.8130
Epoch 47/50
 1536/50000 [..............................] - ETA: 3:50 - loss: 0.1761 - acc: 0.9434

50000/50000 [==============================] - 255s 5ms/step - loss: 0.2064 - acc: 0.9269 - val_loss: 0.6501 - val_acc: 0.8430
Epoch 48/50
38528/50000 [======================>.......] - ETA: 54s - loss: 0.1962 - acc: 0.9298

50000/50000 [==============================] - 254s 5ms/step - loss: 0.2008 - acc: 0.9280 - val_loss: 0.7638 - val_acc: 0.8201
Epoch 49/50
50000/50000 [==============================] - 254s 5ms/step - loss: 0.1986 - acc: 0.9283 - val_loss: 0.5937 - val_acc: 0.8537
Epoch 50/50
 1536/50000 [..............................] - ETA: 3:51 - loss: 0.2040 - acc: 0.9225

50000/50000 [==============================] - 255s 5ms/step - loss: 0.1974 - acc: 0.9296 - val_loss: 0.5955 - val_acc: 0.8560


In [90]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 23s 2ms/step
Test loss: 0.5954634487867355
Test accuracy: 0.856


In [91]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')